In [2]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="devashish",
  database="university"
)

print(mydb)
mycursor = mydb.cursor(buffered=True)


In [3]:
import mysql.connector.cursor


def printall(cursor: mysql.connector.cursor):
    if cursor is None:
        print("curson is None")
        return
    if cursor.column_names is not None: print(cursor.column_names)
    try:
        fetchall = cursor.fetchall()
        if fetchall is None:
            raise Exception
    except Exception as e:
        print(f"Cursor fetched None {e}")
        return
    for row in fetchall:
        print(row)

In [4]:

mycursor.execute("show tables")
printall(mycursor)

('Tables_in_university',)
('advisor',)
('classroom',)
('concept',)
('course',)
('department',)
('event_log',)
('instructor',)
('prereq',)
('section',)
('student',)
('takes',)
('teaches',)
('time_slot',)


In [5]:
mycursor.execute("select * from student limit 10;")
printall(mycursor)

# mycursor.execute("""\
#                  select student.ID as Stud_ID, takes.ID as Takes_ID, name, course_id, semester, year, grade
#                   from student join takes ON student.ID = takes.ID limit 10;
#                  """)
# printall(mycursor)




('ID', 'name', 'dept_name', 'tot_cred', 'timestamp')
('00128', 'Zhang', 'Comp. Sci.', Decimal('102'), datetime.datetime(2009, 4, 1, 8, 0))
('12345', 'Shankar', 'Comp. Sci.', Decimal('32'), datetime.datetime(2009, 4, 1, 8, 0))
('19991', 'Brandt', 'History', Decimal('80'), datetime.datetime(2009, 4, 1, 8, 0))
('23121', 'Chavez', 'Finance', Decimal('110'), datetime.datetime(2009, 4, 1, 8, 0))
('44553', 'Peltier', 'Physics', Decimal('56'), datetime.datetime(2009, 4, 1, 8, 0))
('45678', 'Levy', 'Physics', Decimal('46'), datetime.datetime(2009, 4, 1, 8, 0))
('54321', 'Williams', 'Comp. Sci.', Decimal('54'), datetime.datetime(2009, 4, 1, 8, 0))
('55739', 'Sanchez', 'Music', Decimal('38'), datetime.datetime(2009, 4, 1, 8, 0))
('70557', 'Snow', 'Physics', Decimal('0'), datetime.datetime(2009, 4, 1, 8, 0))
('76543', 'Brown', 'Comp. Sci.', Decimal('58'), datetime.datetime(2009, 4, 1, 8, 0))


In [ ]:
# Add timestamps to student table, to indicate enrollment

# mycursor.execute("ALTER TABLE student DROP COLUMN timestamp")
# mycursor.execute("""ALTER TABLE student ADD COLUMN timestamp DATETIME NOT NULL DEFAULT '2009-04-01 08:00:00' AFTER tot_cred""")
# mycursor.execute("select * from student;")
# printall(mycursor)

In [ ]:
# Add timestamps to takes table to indicate course gradation
# mycursor.execute("""SELECT * FROM takes""")
# printall(mycursor)
# mycursor.execute("""ALTER TABLE takes ADD COLUMN timestamp DATETIME NOT NULL DEFAULT '2010-10-01 20:30:00' AFTER grade""")
mycursor.execute("""UPDATE takes
                 SET timestamp = '2010-03-31 18:00:00' 
                 WHERE semester = 'Fall' and year = 2009""")
mycursor.execute("""UPDATE takes
                 SET timestamp = '2011-03-31 18:00:00' 
                 WHERE semester = 'Fall' and year = 2010""")
mycursor.execute("""UPDATE takes
                 SET timestamp = '2009-10-31 18:00:00' 
                 WHERE (semester = 'Spring' or semester = 'Summer') and year = 2009""")
mycursor.execute("""UPDATE takes
                 SET timestamp = '2010-10-31 18:00:00' 
                 WHERE (semester = 'Spring' or semester = 'Summer') and year = 2010""")
mycursor.execute("""SELECT * FROM takes""")
printall(mycursor)


In [ ]:
# Create concept table
# mycursor.execute("DROP TABLE IF EXISTS concept")
# mycursor.execute("""\
#                     CREATE TABLE concept (
#                     ID varchar(20) NOT NULL,
#                     type varchar(20) NOT NULL,
#                     PRIMARY KEY (ID, type)
#                     )
#                  """)
mycursor.execute("select * from concept")
printall(mycursor)

In [ ]:
# mycursor.execute("""INSERT INTO concept
#                  SELECT DISTINCT ID, 'student' 
#                  from student""")


In [ ]:
# mycursor.execute("""INSERT INTO concept
#                  SELECT DISTINCT ID, 'instructor' 
#                  from instructor""")

In [ ]:
# mycursor.execute("""INSERT INTO concept
#                  SELECT DISTINCT sec_id, 'section' 
#                  from section""")

In [ ]:
# mycursor.execute("""INSERT INTO concept
#                  SELECT DISTINCT dept_name, 'department' 
#                  from department""")

In [ ]:
# mydb.commit()

In [ ]:
# Create OCEL table
mycursor.execute("DROP TABLE IF EXISTS event_log")
mycursor.execute("""CREATE TABLE event_log (
                 event_id INT NOT NULL AUTO_INCREMENT,
                 activity VARCHAR(20) NOT NULL,
                 timestamp DATETIME NOT NULL DEFAULT '2010-10-01 00:30:00',
                 student_id VARCHAR(20),
                 instructor_id VARCHAR(20),
                 section_id varchar(20),
                 course_id varchar(20),
                 department_id VARCHAR(20),
                 PRIMARY KEY (event_id)
                 )
                 """)
mycursor.execute("""SELECT * FROM event_log""")
printall(mycursor)

In [ ]:
# Create Student<>Course Events
# Course Enrolled
# Course Passed
# Course Failed

# Course_enrolled v1
# mycursor.execute("""INSERT INTO
#                  event_log (activity, timestamp, student_id, section_id, course_id)
#                  SELECT 'course_enrolled', '2009-11-01 08:30:00', ID, sec_id, course_id
#                  FROM takes
#                  WHERE semester = 'Fall' and year = 2009""")

mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_enrolled', '2009-11-01 08:30:00', ID, sec_id, course_id,
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE semester = 'Fall' and year = 2009""")
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_enrolled', '2010-11-01 08:30:00', ID, sec_id, course_id,
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE semester = 'Fall' and year = 2010""")
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_enrolled', '2009-04-01 08:30:00', ID, sec_id, course_id,
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE (semester = 'Spring' or semester = 'Summer') and year = 2009""")
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_enrolled', '2010-04-01 08:30:00', ID, sec_id, course_id,
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE (semester = 'Spring' or semester = 'Summer') and year = 2010""")
# mycursor.execute("""SELECT * FROM event_log where event_id > 9 and event_id < 20""")
mycursor.execute("""SELECT count(*) FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log ORDER BY event_id DESC Limit 5""")
printall(mycursor)

In [ ]:
# Course Passed v1
# mycursor.execute("""INSERT INTO
#                  event_log (activity, timestamp, student_id, section_id, course_id,)
#                  SELECT 'course_passed', timestamp, ID, sec_id, course_id
#                  FROM takes
#                  WHERE grade != 'F' AND grade IS NOT NULL""")
# Course Passed v2
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_passed', timestamp, ID, sec_id, course_id, 
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE grade != 'F' AND grade IS NOT NULL""")
# Course Failed
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, section_id, course_id, department_id, instructor_id)
                 SELECT 'course_failed', timestamp, ID, sec_id, course_id, 
                 (select dept_name from course where takes.course_id = course.course_id),
                 (select teaches.ID from teaches where takes.course_id = teaches.course_ID AND takes.sec_id = teaches.sec_id AND takes.semester = teaches.semester AND takes.year = teaches.year)
                 FROM takes
                 WHERE grade = 'F'""")
# mycursor.execute("""SELECT * FROM event_log where event_id > 9 and event_id < 20""")
mycursor.execute("""SELECT count(*) FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log ORDER BY event_id DESC Limit 5""")
printall(mycursor)

In [ ]:
# advisor_assigned
# mycursor.execute("""ALTER TABLE advisor ADD COLUMN timestamp DATETIME NOT NULL DEFAULT '2009-04-30 08:30:00' AFTER i_ID""")

mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, instructor_id)
                 SELECT 'advisor_assigned', timestamp, s_ID, i_ID
                 FROM advisor
                 """)

In [ ]:
# Student enrolled
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, department_id)
                 SELECT 'student_enrolled', timestamp, student.ID, dept_name
                 FROM student
                 """)
# Student graduated
# This query is techincally incorrrect because you should choose the timestamp which has the latest value,
# but we only choose one distinct timestamp because the mocked timestamps are equal
mycursor.execute("""INSERT INTO
                 event_log (activity, timestamp, student_id, department_id, instructor_id)
                 SELECT DISTINCT 'student_graduated', takes.timestamp, student.ID, student.dept_name,
                 (select i_ID from advisor where student.ID = advisor.s_ID)
                 FROM student JOIN takes
                 ON student.ID = takes.ID
                 WHERE tot_cred >= 100 AND grade != 'F' and grade IS NOT NULL

                 """)
mycursor.execute("""SELECT count(*) FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log WHERE activity = 'student_graduated' ORDER BY event_id DESC""")
printall(mycursor)

In [ ]:
mycursor.execute("""SELECT count(*) FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT DISTINCT activity FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log""")
printall(mycursor)
mycursor.execute("""SELECT count(*) FROM concept""")
printall(mycursor)
mycursor.execute("""SELECT * FROM concept""")
printall(mycursor)

In [ ]:
mycursor.execute("""SELECT count(*) FROM event_log where student_id = '00128'""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log where student_id = '00128' ORDER BY timestamp ASC""")
printall(mycursor)

In [ ]:
mycursor.execute("""SELECT count(*) FROM event_log where instructor_id = '45565'""")
printall(mycursor)
mycursor.execute("""SELECT * FROM event_log where instructor_id = '45565' ORDER BY timestamp ASC""")
printall(mycursor)

In [ ]:
# mydb.commit()